In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tltorch
import torch, torchvision
import numpy as np

### Tensorly Torch TRL Example

In [5]:
input_shape = (5, 3, 2)
output_shape = (5,2)
batch_size = 1

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device='cpu'
print(device)

x = torch.randn((batch_size,) + input_shape,
                dtype=torch.float32, device=device)
trl = tltorch.TRL(input_shape, output_shape, rank='same')
result = trl(x)
print(x.shape, result.shape)

cpu
torch.Size([1, 5, 3, 2]) torch.Size([1, 5, 2])


Look's like TRL let's you reshape the tensor along all modes. 

### CNN

In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 32
num_classes = 10
learning_rate = 1e-3
num_epochs = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

##### Download and preprocess CIFAR10 dataset

In [6]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.trl = tltorch.TRL(input_shape=(64, 5, 5), output_shape=(32, 6, 6), rank='same')
        self.fc1 = nn.Linear(32*6*6, 128) # 128 is a random choice by me could be anything 

        # self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = self.trl(out)
                
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
model = ConvNeuralNet(num_classes)
if torch.cuda.is_available():
    model.cuda()
# from torchsummary import summary
# print(summary(model, (3,32,32)))
print(model)

ConvNeuralNet(
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (trl): TRL(
    (weight): CPTensor(shape=(64, 5, 5, 32, 6, 6), rank=15620)
  )
  (fc1): Linear(in_features=1152, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
def train(num_epochs, model, optimizer, lossfn):
    losses = []
    for epoch in range(num_epochs):
        epoch_loss = []
        #Load in the data in batches using the train_loader object
        for i, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = lossfn(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            epoch_loss.append(loss.item())
            optimizer.step()

        losses.append(sum(epoch_loss)/len(epoch_loss))
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    return losses

lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  
total_step = len(train_loader)
losses = train(num_epochs, model, optimizer, lossfn)

Epoch [1/2], Loss: 1.7722
Epoch [2/2], Loss: 0.7808
